In [20]:
import numpy as np
from lab1.Lab1_FK_answers import *

In [4]:
bvh = open("../data/walk60.bvh", 'r')

In [5]:
line = bvh.readline()
while line != "MOTION\n":
    line = bvh.readline()

In [11]:
line = bvh.readline()

In [12]:
line

' -0.001735   0.855388   0.315499   2.008551   7.606260  -0.798294  11.216058  -3.286777  -1.592436  13.521250  -1.153514  -4.213484 -17.754157  -3.216621   9.232892  -7.948705   0.211932  -1.528529   2.220789  -0.981058  -1.133630   2.071938  -6.311876   2.083844   2.020309  -0.533885 -19.342332  -5.129554 -37.575293 -50.190804   0.198025 -24.741038   4.442069   0.442380   2.547494   4.858004   1.951773  -5.809334  21.100535  23.710456  30.003467  53.240376   0.414981  10.414544   1.952633   3.576914  -9.482057   6.918939   1.457480  -0.035296   0.111891 -27.722826  -1.655032   2.430426  -2.964232  -5.507982   1.444119   2.239212  -3.180259  -0.892285  -0.008100  -0.007000   0.024400\n'

In [14]:
line.split()

['-0.001735',
 '0.855388',
 '0.315499',
 '2.008551',
 '7.606260',
 '-0.798294',
 '11.216058',
 '-3.286777',
 '-1.592436',
 '13.521250',
 '-1.153514',
 '-4.213484',
 '-17.754157',
 '-3.216621',
 '9.232892',
 '-7.948705',
 '0.211932',
 '-1.528529',
 '2.220789',
 '-0.981058',
 '-1.133630',
 '2.071938',
 '-6.311876',
 '2.083844',
 '2.020309',
 '-0.533885',
 '-19.342332',
 '-5.129554',
 '-37.575293',
 '-50.190804',
 '0.198025',
 '-24.741038',
 '4.442069',
 '0.442380',
 '2.547494',
 '4.858004',
 '1.951773',
 '-5.809334',
 '21.100535',
 '23.710456',
 '30.003467',
 '53.240376',
 '0.414981',
 '10.414544',
 '1.952633',
 '3.576914',
 '-9.482057',
 '6.918939',
 '1.457480',
 '-0.035296',
 '0.111891',
 '-27.722826',
 '-1.655032',
 '2.430426',
 '-2.964232',
 '-5.507982',
 '1.444119',
 '2.239212',
 '-3.180259',
 '-0.892285',
 '-0.008100',
 '-0.007000',
 '0.024400']

In [15]:
len(line.split())

63

In [17]:
306 - 125 + 2

183

In [21]:
bvh_path = "../data/walk60.bvh"

In [24]:
load_motion_data(bvh_path)[0, :]

array([-1.7350000e-03,  8.5538800e-01,  3.1549900e-01,  2.0085510e+00,
        7.6062600e+00, -7.9829400e-01,  1.1216058e+01, -3.2867770e+00,
       -1.5924360e+00,  1.3521250e+01, -1.1535140e+00, -4.2134840e+00,
       -1.7754157e+01, -3.2166210e+00,  9.2328920e+00, -7.9487050e+00,
        2.1193200e-01, -1.5285290e+00,  2.2207890e+00, -9.8105800e-01,
       -1.1336300e+00,  2.0719380e+00, -6.3118760e+00,  2.0838440e+00,
        2.0203090e+00, -5.3388500e-01, -1.9342332e+01, -5.1295540e+00,
       -3.7575293e+01, -5.0190804e+01,  1.9802500e-01, -2.4741038e+01,
        4.4420690e+00,  4.4238000e-01,  2.5474940e+00,  4.8580040e+00,
        1.9517730e+00, -5.8093340e+00,  2.1100535e+01,  2.3710456e+01,
        3.0003467e+01,  5.3240376e+01,  4.1498100e-01,  1.0414544e+01,
        1.9526330e+00,  3.5769140e+00, -9.4820570e+00,  6.9189390e+00,
        1.4574800e+00, -3.5296000e-02,  1.1189100e-01, -2.7722826e+01,
       -1.6550320e+00,  2.4304260e+00, -2.9642320e+00, -5.5079820e+00,
      

In [ ]:
def orientations(joint_parents: List[int],
                 joint_offsets: np.ndarray,
                 motion_data_one_frame: np.ndarray) -> np.ndarray:
    joint_orientations = []
    
    
    return np.array(joint_orientations, dtype=np.float64)
    

In [ ]:
def part2_forward_kinematics(joint_names, joint_parents, joint_offsets, motion_data, frame_id):
    """请填写以下内容
    输入: part1 获得的关节名字，父节点列表，偏移量列表
        motion_data: np.ndarray，形状为(N,X)的numpy数组，其中N为帧数，X为Channel数
        frame_id: int，需要返回的帧的索引
    输出:
        joint_positions: np.ndarray，形状为(M, 3)的numpy数组，包含着所有关节的全局位置
        joint_orientations: np.ndarray，形状为(M, 4)的numpy数组，包含着所有关节的全局旋转(四元数)
    Tips:
        1. joint_orientations的四元数顺序为(x, y, z, w)
        2. from_euler时注意使用大写的XYZ
    """
    joint_orientations = orientations(joint_parents, joint_offsets, motion_data[frame_id])

    joint_positions = None
    return joint_positions, joint_orientations